<a href="https://colab.research.google.com/github/ClaytonLucas/m-trica-avalia-o-aprendizado/blob/main/calculo_matriz_de_confus%C3%A3o.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q tensorflow-gpu==2.0.0-beta1

ERROR: Could not find a version that satisfies the requirement tensorflow-gpu==2.0.0-beta1 (from versions: 2.12.0)
ERROR: No matching distribution found for tensorflow-gpu==2.0.0-beta1


In [ ]:
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
import tensorflow as tf

import numpy as np

import seaborn as sns

import pandas as pd

In [ ]:
tf.__version__

'2.17.1'

In [ ]:
%load_ext tensorboard

In [ ]:
logdir='log'

In [ ]:
(train_images, train_labels), (test_images, test_labels) = datasets.mnist.load_data()

train_images = train_images.reshape((60000, 28, 28, 1))
test_images = test_images.reshape((10000, 28, 28, 1))

train_images, test_images = train_images / 255.0, test_images / 255.0

classes=[0,1,2,3,4,5,6,7,8,9]

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

In [ ]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x=train_images,
            y=train_labels,
            epochs=5,
            validation_data=(test_images, test_labels))

Epoch 1/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 65s 34ms/step - accuracy: 0.8981 - loss: 0.3309 - val_accuracy: 0.9836 - val_loss: 0.0484
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 80s 33ms/step - accuracy: 0.9853 - loss: 0.0468 - val_accuracy: 0.9888 - val_loss: 0.0345
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 62s 33ms/step - accuracy: 0.9897 - loss: 0.0328 - val_accuracy: 0.9890 - val_loss: 0.0334
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 79s 31ms/step - accuracy: 0.9922 - loss: 0.0246 - val_accuracy: 0.9905 - val_loss: 0.0290
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 85s 33ms/step - accuracy: 0.9939 - loss: 0.0183 - val_accuracy: 0.9891 - val_loss: 0.0366


In [ ]:
y_true=test_labels
y_pred=(model.predict(test_images) > 0.5).astype("int32")

313/313 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step


In [ ]:
classes=[0,1,2,3,4,5,6,7,8,9]

In [ ]:
con_mat = tf.math.confusion_matrix(labels=y_true, predictions=y_pred).numpy()
con_mat_norm = np.around(con_mat.astype('float') / con_mat.sum(axis=1)[:, np.newaxis], decimals=2)

con_mat_df = pd.DataFrame(con_mat_norm,
                     index = classes,
                     columns = classes)

figure = plt.figure(figsize=(8, 8))
sns.heatmap(con_mat_df, annot=True,cmap=plt.cm.Blues)
plt.tight_layout()
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()

In [ ]:
sensibilidade = []
especificidade = []
precisao = []
f1_score = []

total_samples = np.sum(con_mat)

In [ ]:
for i in range(len(classes)):
    VP = con_mat[i, i]
    FN = np.sum(con_mat[i, :]) - VP
    FP = np.sum(con_mat[:, i]) - VP
    VN = total_samples - (VP + FN + FP)


    sensibilidade_classe = VP / (VP + FN) if (VP + FN) != 0 else 0
    sensibilidade.append(sensibilidade_classe)


    especificidade_classe = VN / (VN + FP) if (VN + FP) != 0 else 0
    especificidade.append(especificidade_classe)


    precisao_classe = VP / (VP + FP) if (VP + FP) != 0 else 0
    precisao.append(precisao_classe)


    f1_classe = (2 * precisao_classe * sensibilidade_classe) / (precisao_classe + sensibilidade_classe) if (precisao_classe + sensibilidade_classe) != 0 else 0
    f1_score.append(f1_classe)


acuracia = np.trace(con_mat) / total_samples

In [ ]:
print("Acurácia (global):", acuracia)
for i, classe in enumerate(classes):
    print(f"Classe {classe}:")
    print(f"  Sensibilidade: {sensibilidade[i]:.2f}")
    print(f"  Especificidade: {especificidade[i]:.2f}")
    print(f"  Precisão: {precisao[i]:.2f}")
    print(f"  F1-Score: {f1_score[i]:.2f}")

In [26]:
!git init
!git config --global user.email "clayton.lucas07@gmail.com"
!git config --global user.name "ClaytonLucas"

Reinitialized existing Git repository in /content/.git/


In [28]:
!git add .
!git commit -m "first commit"


On branch main
nothing to commit, working tree clean
Enumerating objects: 28, done.
Counting objects: 100% (28/28), done.
Delta compression using up to 2 threads
Compressing objects: 100% (21/21), done.
Writing objects: 100% (28/28), 8.42 MiB | 2.06 MiB/s, done.
Total 28 (delta 5), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (5/5), done.
To https://github.com/ClaytonLucas/m-trica-avalia-o-aprendizado.git
 * [new branch]      main -> main


In [ ]:
!git add .